# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120120e+02     1.585256e+00
 * time: 0.13991785049438477
     1     1.102986e+01     1.016311e+00
 * time: 1.0087878704071045
     2    -1.193048e+01     1.097339e+00
 * time: 1.1447618007659912
     3    -3.385283e+01     8.313460e-01
 * time: 1.3119189739227295
     4    -4.732927e+01     6.699657e-01
 * time: 1.4664809703826904
     5    -5.686932e+01     2.201666e-01
 * time: 1.6147289276123047
     6    -5.967720e+01     1.783789e-01
 * time: 1.7229368686676025
     7    -6.082629e+01     6.338798e-02
 * time: 1.8575890064239502
     8    -6.132566e+01     4.854591e-02
 * time: 1.9649109840393066
     9    -6.160566e+01     4.750617e-02
 * time: 2.079838991165161
    10    -6.179305e+01     3.086306e-02
 * time: 2.188450813293457
    11    -6.194396e+01     2.425773e-02
 * time: 2.3037469387054443
    12    -6.203679e+01     1.967582e-02
 * time: 2.4109139442443848
    13    -6.208223e+01     1.664681e-02
 * time: 2.52426695823

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557706
    AtomicNonlocal      14.8522671
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336821

    total               -62.261666459817
